In [1]:
import os
import sys
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta

from stndata import DAILYSTNNAMES, DAILYDTYPE, DAILYNAMES
from stndata import WXCODES

TZ = {
    "QLD": 10,
    "NSW": 10,
    "VIC": 10,
    "TAS": 10,
    "SA": 9.5,
    "NT": 9.5,
    "WA": 8,
    "ANT": 0,
}

# List of columns to be used in the merge with the storm database:
COLUMNS = ['datetimeUTC', 'datetimeLST', 'date', 'stnNum',
           'PresentWxCode', 'PastWxCode',
           'thunder', 'hail', 'duststorm']

In [2]:
BASEPATH = os.path.dirname(os.getcwd())
DATAPATH = os.path.join(BASEPATH, 'data')
DAILYMAXPATH = os.path.join(DATAPATH, "allevents", "dailymax")
GUSTRATIOPATH = os.path.join(DATAPATH, "training", "gustratio")
OUTPUTPATH = os.path.join(DATAPATH, "allevents", "results")

In [3]:
def findPresentWx(row):
    """
    Find the present weather code (if reported) and return the weather description.

        Usage:

        df.apply(findPresentWx, axis=1)

    :param row: Row of a DataFrame that includes a datetime and present weather 
    codes
    :return: Weather description (if the code exists), or empty string
    :rtype: string
    """
    hour = row['datetimeLST'].hour
    nearest_hour = min(range(0, 24, 3), key=lambda x: abs(x - hour))
    nearest_presentWx_column = f'PresentWx{str(nearest_hour).zfill(2)}'
    try:
        code = int(row[nearest_presentWx_column])
        return WXCODES[code]
    except ValueError:
        return row[nearest_presentWx_column]


def findPastWx(row):
    """
    Find the past weather code that corresponds to the time of the maximum
    daily gust (if reported) and return the weather description

    Usage:

        df.apply(findPastWx, axis=1)

    :param row: Row of a DataFrame
    :return: Weather description (if the code exists) or empty string
    :rtype: string
    """
    hour = row['datetimeLST'].hour
    pastWx_hours = [int(col[6:8]) for col in row.index if col.startswith('PastWx')]
    # Find the next PastWx hour
    next_hour = min(filter(lambda x: x > hour, pastWx_hours), default=min(pastWx_hours))
    
    next_pastWx_column = f'PastWx{str(next_hour).zfill(2)}'
    try:
        code = int(row[next_pastWx_column])
        return WXCODES[code]
    except ValueError:
        return row[next_pastWx_column]

In [4]:
def loadDailyData(filename: str, stnState: str) -> pd.DataFrame:
    """
    Load a daily max wind gust file, determine the present and past weather code (if they are recorded) and return a `pd.DataFrame`

    :param filename: _description_
    :type filename: str
    :param stnState: _description_
    :type stnState: str
    :return: _description_
    :rtype: pd.DataFrame
    """
    try:
        df = pd.read_csv(filename,sep=",",
                    index_col=False,
                    names=DAILYNAMES,
                    header=0,
                    parse_dates={"datetimeLST": [2, 3, 4, 15]},
                    keep_date_col=True,
                    date_format="%Y %m %d %H%M",
                    na_values=["####"],
                skipinitialspace=True,)
    except FileNotFoundError:
        LOGGER.exception(f"Cannot load {filename} - cannot find the file")

    # Remove any records where the time is not indicated:
    df.dropna(axis=0, subset=['HHMM'], inplace=True)
    
    # Convert time data
    df['datetimeLST'] = pd.to_datetime(df['datetimeLST'],
                                       format="%Y %m %d %H%M")
    df["datetimeUTC"] = df.datetimeLST - timedelta(hours=TZ[stnState])
    df['datetimeUTC'] = df.datetimeUTC.dt.tz_localize("UTC")
    df["date"] = df.datetimeUTC.dt.date

    # Extract the weather codes:
    df['PresentWeatherCode'] = df.apply(findPresentWx, axis=1)
    df['PastWeatherCode'] = df.apply(findPastWx, axis=1)
    return df

In [11]:
stnNum = 4032
stnState = "WA"

dailyStnFile = rf"X:\georisk\HaRIA_B_Wind\data\raw\from_bom\2023\daily\DC02D_Data_{stnNum:06d}_9999999910405892.txt"
dailyStnData = pd.read_csv(dailyStnFile,sep=",",
            index_col=False,
            #dtype=DAILYDTYPE,
            names=DAILYNAMES,
            header=0,
            parse_dates={"datetimeLST": [2, 3, 4, 15]},
            keep_date_col=True,
            date_format="%Y %m %d %H%M",
            na_values=["####"],
            skipinitialspace=True,)
# Remove any records where the time is not indicated:
dailyStnData.dropna(axis=0, subset=['HHMM'], inplace=True)
dailyStnData['datetimeLST'] = pd.to_datetime(dailyStnData['datetimeLST'], format="%Y %m %d %H%M")
dailyStnData["datetimeUTC"] = dailyStnData.datetimeLST - timedelta(hours=TZ[stnState])
dailyStnData['datetimeUTC'] = dailyStnData.datetimeUTC.dt.tz_localize("UTC")
dailyStnData["date"] = dailyStnData.datetimeUTC.dt.date

dailyStnData['PresentWxCode'] = dailyStnData.apply(findPresentWx, axis=1)
dailyStnData['PastWxCode'] = dailyStnData.apply(findPastWx, axis=1)

C:\Users\u12161\AppData\Local\Temp\ipykernel_24092\3072478875.py:5: DtypeWarning: Columns (18,20,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  dailyStnData = pd.read_csv(dailyStnFile,sep=",",


In [6]:
dailyStnData = dailyStnData[COLUMNS]

In [7]:
stormData = pd.read_csv(os.path.join(OUTPUTPATH, "storm_classification_data.csv"))

In [8]:
stormData['datetimeUTC'] = pd.to_datetime(stormData.datetimeUTC)
stormData['datetimeLST'] = pd.to_datetime(stormData.datetimeLST)

In [9]:
stormData.merge(dailyStnData, how='left', on=['datetimeUTC', 'stnNum']).columns

Index(['datetimeUTC', 'datetimeLST_x', 'stnNum', 'windgust', 'windspeed',
       'winddir', 'temperature', 'dewpoint', 'rainfallPre', 'rainfallPost',
       'tempPre', 'tempPost', 'meanSLP', 'stnPressure', 'gustRatio',
       'emergence', 'stormType', 'category', 'datetimeLST_y', 'date',
       'PresentWxCode', 'PastWxCode', 'thunder', 'hail', 'duststorm'],
      dtype='object')

In [12]:
dailyStnData.columns

Index(['datetimeLST', 'id', 'stnNum', 'YYYY', 'MM', 'DD', 'tmax', 'tmaxq',
       'tmaxdays', 'tmin', 'tminq', 'tmindays', 'windgust', 'windgustq',
       'winddir', 'winddirq', 'HHMM', 'timeq', 'PresentWx00', 'PresentWx00Q',
       'PresentWx03', 'PresentWx03Q', 'PresentWx06', 'PresentWx06Q',
       'PresentWx09', 'PresentWx09Q', 'PresentWx12', 'PresentWx12Q',
       'PresentWx15', 'PresentWx15Q', 'PresentWx18', 'PresentWx18Q',
       'PresentWx21', 'PresentWx21Q', 'PastWx00', 'PastWx00Q', 'PastWx03',
       'PastWx03Q', 'PastWx06', 'PastWx06Q', 'PastWx09', 'PastWx09Q',
       'PastWx12', 'PastWx12Q', 'PastWx15', 'PastWx15Q', 'PastWx18',
       'PastWx18Q', 'PastWx21', 'PastWx21Q', 'duststorm', 'duststormq', 'snow',
       'snowq', 'haze', 'hazeq', 'hail', 'hailq', 'fog', 'fogq', 'thunder',
       'thunderq', 'frost', 'frostq', 'end', 'datetimeUTC', 'date',
       'PresentWxCode', 'PastWxCode'],
      dtype='object')